In [1]:
# Add spacing, origin and subset metadata to the HDF5 dataset
import pandas as pd
from pathlib import Path
import SimpleITK as sitk
from tqdm import tqdm
import h5py

/home/ofont/miniconda3/envs/lucanode/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [22]:
files = [f for f in Path("/mnt/Data/LUNA/dataset").glob("subset*/1.*.mhd")]

In [23]:
def get_subset(f):
    return int(str(f).split("/")[-2][-1])
def get_id(f):
    return str(f).split("/")[-1].split(".mhd")[0]
def get_origin(f):
    img = sitk.ReadImage(str(f))
    return img.GetOrigin()

In [32]:
raw_df = [(get_id(f), get_subset(f), get_origin(f)) for f in tqdm(files)]

100%|██████████| 888/888 [10:07<00:00,  1.46it/s]


In [43]:
f = h5py.File("/mnt/Data/LUNA/dataset_05mm.hdf5", "r+")
for metadata in tqdm(raw_df):
    seriesuid = metadata[0]
    subset = metadata[1]
    origin = metadata[2]
    spacing = (0.5, 0.5, 0.5)
    for g in f.keys():
        if seriesuid not in f[g]:
            continue
        f[g][seriesuid].attrs["subset"] = subset
        f[g][seriesuid].attrs["origin"] = origin
        f[g][seriesuid].attrs["spacing"] = spacing
f.close()

100%|██████████| 888/888 [01:13<00:00, 12.12it/s]
